<a href="https://colab.research.google.com/github/Storkluc/Python/blob/master/Cost_of_marketplace_B2W_and_site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Bloco de Inicialização

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt


timestr = time.strftime('%m-%Y')


#Bloco de funções

#Substituir estoque
def changeStore(status):
  if status == 'Disponível':
    status = 50
    return status
  elif status == 'Indisponível':
    status = 0
    return status
  elif status == 'Crítico':
    status = 2
    return status
  else: 
    status = 0
    return status

#Calcula o primeiro custo com impostos PR
def inicialCostPR(pricePR, ipi, st):
    precoImpostoPR = 0
    precoImpostoPR = pricePR + ((pricePR*(ipi/100) + pricePR*(st/100)))
    return precoImpostoPR

#Calcula o primeiro custo com impostos SC
def inicialCostSC(priceSC, ipi):
  precoImpostoSC = 0
  precoImpostoSC = priceSC + (priceSC*(ipi/100))
  return precoImpostoSC

def changeData(data):
  data = data.replace(',', '.')
  data = pd.to_numeric(data)
  return data

def changeData2(celula):
  try:
       celula.fillna(int(0), inplace=True)
  except AttributeError:
    if (pd.isnull(celula)):
      return int(0) 
  return celula

#Calcula o custo pós descontos
def calculateCost(valor,desconto,descontoPAC):
  valorFinal = (valor-(valor*(desconto)+(valor*(descontoPAC))))
  return valorFinal

#Calcula o custo pós descontos
def calculateCostBlack(valor,desconto,descontoPAC,descontoBlack):
  valorFinal = (valor-(valor*(desconto)+(valor*(descontoPAC)+(valor*(descontoBlack)))))
  return valorFinal

#Compara custo final PR x SC
def comparaCusto(estoquePR, estoqueSC, custoPR, custoSC):
  menorCusto = 0
  if estoquePR >= estoqueSC and estoquePR > 0:
    if custoPR < custoSC:
      menorCusto = custoPR
      return menorCusto
    else:
      menorCusto = custoSC
  if estoquePR == 0:
      menorCusto = custoSC
      return menorCusto    
  if estoqueSC >= estoquePR and estoqueSC > 0:
    if custoSC < custoPR:
      menorCusto = custoSC
      return menorCusto
    else:
      menorCusto = custoPR
      return menorCusto
  if estoqueSC == 0:
    menorCusto = custoPR
    return menorCusto
  else:
    return menorCusto

#Identifica a região do menor custo
def menorCusto(custoPR, custoSC, menorCusto):
  estado = ''
  if custoSC == menorCusto:
      estado = 'SC'
  else:
      estado = 'PR'
  return estado

#Calcula Preço de venda do Site
def precoSite(valorS):
  valorSite = valorS+(valorS*0.35)
  return valorSite

#Calcula Preço de venda do Mercado Livre Class
def precoMLClass(valorMC):
  valorMLClass = valorMC+(valorMC*0.46)
  return valorMLClass

#Calcula Preço de venda do Mercado Livre Premium
def precoMLPremium(valorMP):
  valorMLPremium = valorMP+(valorMP*0.08)
  return valorMLPremium

#Calcula Preço de venda da B2W
def precoB2W(valorB2W, freteB2W):
  if valorB2W > 0:
    valorB2W = valorB2W + freteB2W
  return valorB2W

#Calcula a comissão de frete ou taxa fixa do produto
def calculaFreteB2W(largura, altura, comprimento, peso, somaDimensoes,pesoCubado, valor):
    if pesoCubado <= 5 or pesoCubado < peso:
     peso = peso
    else:
      peso = pesoCubado
    if largura and altura and comprimento < 90 or somaDimensoes < 200 and valor > 0:
      if valor < 39.99:
        taxaFrete = 5
        somataxa = valor + taxaFrete
        if somataxa > 39.99:
          taxaFrete = 12.90
        return taxaFrete
      elif valor < 79.99 and peso < 28.999:
        taxaFrete = 12.90
        return taxaFrete
      if valor > 80.00:
        if peso < 0.499:
          taxaFrete = 32.90
          return taxaFrete
        elif peso < 0.999:
          taxaFrete = 35.90
          return taxaFrete
        elif peso < 1.999:
          taxaFrete = 36.90
          return taxaFrete
        elif peso < 4.999:
          taxaFrete = 45.90
          return taxaFrete
        elif peso < 8.999:
          taxaFrete = 67.90
          return taxaFrete
        elif peso < 12.999:
          taxaFrete = 91.90
          return taxaFrete
        elif peso < 16.999:
          taxaFrete = 104.90
          return taxaFrete
        elif peso < 22.999:
          taxaFrete = 112.90
          return taxaFrete
        elif peso < 28.999:
          taxaFrete = 119.90
          return taxaFrete
        else: taxaFrete = 5
        return taxaFrete  
    elif valor > 0 and valor > 5:
     taxaFrete = 5
     return taxaFrete


print('\n\nExecutado com sucesso')
#Calcula a soma das dimensões dos produtos
def takeDimensoes(larguraP, alturaP, comprimentoP):
  dimensoesTotais = larguraP + alturaP + comprimentoP
  return dimensoesTotais

#Calcula o peso cubado dos produtos
def takeCubo(larguraC, alturaC, comprimentoC):
  pesoCubado = (larguraC * alturaC * comprimentoC)/6000
  return pesoCubado

#Arruma estoque após classificação por filial
def fixStore(estPR, estSC, valorPR, valorSC, menorValor):
  fixstore = 0
  if menorValor == valorSC:
    fixstore = estSC
    return fixstore
  if menorValor == valorPR:
    fixstore = estPR
    return fixstore

#Define o maior peso
def heavierWeight(pesoProduto, cubado, somadimensoes, larguraProduto, comprimentoProduto, alturaProduto, valorProduto):
  resultado = ''
  if pesoProduto and cubado < 29:
    if larguraProduto and alturaProduto and comprimentoProduto < 90 and somadimensoes < 200 and valorProduto > 0 and valorProduto < 10000:
      resultado = 'B2W Entregas'
    else:
      resultado = 'Transportadora'
  else:
    resultado = 'Transportadora'
  return resultado




Executado com sucesso


###Bloco de geração de relatórios

In [ ]:
#Gera template planilha Custo Inicial PR
template_custo_inicial_PR = pd.DataFrame(columns=['SKU','DESCRIÇÃO','CUSTO','IPI','%ST', 'ESTOQUE'],dtype=int)
template_custo_inicial_PR.to_excel('custo_PR_' + timestr + '.xls', index=False)

#Gera template planilha Custo Inicial SC
template_custo_inicial_SC = pd.DataFrame(columns=['SKU','DESCRIÇÃO','CUSTO','IPI', 'ESTOQUE'],dtype=int)
template_custo_inicial_SC.to_excel('custo_SC_' + timestr + '.xls',  index=False)

#Gera planilha template para o desconto
template_custo_inicial_PR = pd.DataFrame(columns=['SKU','DESCRIÇÃO','DESCONTO'],dtype=int)
template_custo_inicial_PR.to_excel('Descontos_fornecedor_' + timestr + '.xls',  index=False)
print('\n\nArquivos Gerados')
#Gera planilha template para desconto PAC
template_custo_inicial_PR = pd.DataFrame(columns=['SKU','DESCRIÇÃO','DESCONTO'],dtype=int)
template_custo_inicial_PR.to_excel('Descontos_PAC_' + timestr + '.xls',  index=False)

#Gera template Frete B2W
template_custo_inicial_PR = pd.DataFrame(columns=['SKU','DESCRIÇÃO','PESO','LARGURA', 'ALTURA','COMPRIMENTO'], dtype=int)
template_custo_inicial_PR.to_excel('dados_produtos_' + timestr + '.xls',  index=False)



Arquivos Gerados


##Bloco calculo do custo

In [ ]:
#Importa o template PR
dfPR = pd.read_excel('/content/custo_PR_'+ timestr + '.xls')

#Importa o template SC
dfSC = pd.read_excel('/content/custo_SC_'+ timestr + '.xls')

#Importa base de cadastros
dfBASE = pd.read_excel('/content/BASEPRODUTOS.xlsx')

#Conversão do estoque em números
dfPR['ESTOQUE'] = dfPR.apply(lambda row: changeStore(row['ESTOQUE']), axis=1)
dfSC['ESTOQUE'] = dfSC.apply(lambda row: changeStore(row['ESTOQUE']), axis=1)

#Convertendo campos vázios por 0
dfPR['%ST'] = dfPR.apply(lambda row: changeData2(row['%ST']), axis=1)
dfPR['IPI'] = dfPR.apply(lambda row: changeData2(row['IPI']), axis=1)
dfSC['IPI'] = dfSC.apply(lambda row: changeData2(row['IPI']), axis=1)

#Calculando o primeiro custo
dfPR['CUSTO IMPOSTO'] = round(dfPR.apply(lambda row: inicialCostPR(row['CUSTO'], row['%ST'], row['IPI']), axis=1),2)
dfSC['CUSTO IMPOSTO'] = round(dfSC.apply(lambda row: inicialCostSC(row['CUSTO'], row['IPI']), axis=1),2)
dfPR.head()

,SKU,DESCRIÇÃO,CUSTO,IPI,%ST,ESTOQUE,CUSTO IMPOSTO
0,6659036055,"ALISADORA DE PISO GASOL.36"" 7,8HP VONDE",6639.27,0.0,0.00,2,6639.27
1,6061350000,ACESSORIOS P/MICRORRETIFICA ARV350 VOND,173.83,8.0,19.08,50,220.90
2,6220013000,AEROGRAFO C/ACESSORIOS 13PCS VONDER,162.10,5.0,0.00,50,170.20
3,6220016000,AEROGRAFO C/ACESSORIOS 16PCS VONDER,314.27,5.0,0.00,50,329.98
4,6220010000,AEROGRAFO C/MINI COMPRESSOR VONDER,307.66,5.0,0.00,50,323.04


In [ ]:
#Criação do dataframe com os dados da duas filiais
dfCustoFinal = dfBASE
dfCustoFinal = pd.merge(dfCustoFinal, dfPR, how='left')
dfCustoFinal = pd.merge(dfCustoFinal, dfSC, how='left')
dfCustoFinal.drop_duplicates(inplace=True,subset=['SKU'])
dfCustoFinal['%ST'] = dfCustoFinal.apply(lambda row: changeData2(row['%ST']), axis=1)
dfCustoFinal.drop('DESCRIÇÃO', axis=1, inplace=True)

#Gerando dataframe de comparação entre filiais
dfCompara = dfCustoFinal[['SKU']]
dfCompara = pd.merge(dfCompara, dfSC[['SKU','ESTOQUE','CUSTO IMPOSTO']], on='SKU', how='left')
dfCompara = pd.merge(dfCompara, dfPR[['SKU','ESTOQUE','CUSTO IMPOSTO']], on='SKU', how='left')
dfCompara.drop_duplicates(inplace=True,subset=['SKU'])
dfCompara.rename(columns={'ESTOQUE_x': 'ESTOQUE SC', 'CUSTO IMPOSTO_x':'CUSTO SC', 'ESTOQUE_y':'ESTOQUE PR', 'CUSTO IMPOSTO_y':'CUSTO PR'}, inplace=True)
dfCompara['ESTOQUE SC'] = dfCompara.apply(lambda row: changeData2(row['ESTOQUE SC']), axis=1)
dfCompara['CUSTO SC'] = dfCompara.apply(lambda row: changeData2(row['CUSTO SC']), axis=1)
dfCompara['ESTOQUE PR'] = dfCompara.apply(lambda row: changeData2(row['ESTOQUE PR']), axis=1)
dfCompara['CUSTO PR'] = dfCompara.apply(lambda row: changeData2(row['CUSTO PR']), axis=1)

#Separando o menor custo entre as filiais
dfCompara['Menor Custo'] = dfCompara.apply(lambda row: comparaCusto(row['ESTOQUE PR'],row['ESTOQUE SC'],row['CUSTO PR'], row['CUSTO SC']), axis=1)
dfCompara['Estado'] = dfCompara.apply(lambda row: menorCusto(row['CUSTO PR'], row['CUSTO SC'], row['Menor Custo']), axis=1)
dfCompara['ESTOQUE Compara'] = dfCompara.apply(lambda row: fixStore(row['ESTOQUE PR'],row['ESTOQUE SC'],row['CUSTO PR'], row['CUSTO SC'], row['Menor Custo']), axis=1)
#dfCompara = dfCompara[['SKU', 'ESTOQUE Compara','Estado', 'Menor Custo']]
dfCustoFinal = pd.merge(dfCustoFinal, dfCompara[['SKU', 'ESTOQUE Compara', 'Menor Custo','Estado']], how='inner', on='SKU')
dfCustoFinal.head(5)

,SKU,Descrição,CUSTO,IPI,%ST,ESTOQUE,CUSTO IMPOSTO,ESTOQUE Compara,Menor Custo,Estado
0,6659036055,"Acabadora de Superfície a Gasolina, 36"", 7,8 H...",6639.27,0.0,0.00,2.0,6639.27,2.0,6639.27,PR
1,6061350000,Acessórios Para Microretífica Arv350 VONDER,173.83,8.0,19.08,50.0,220.90,50.0,172.09,SC
2,6220013000,Aerógrafo com Acessórios 13 Peças VONDER,162.10,5.0,0.00,50.0,170.20,50.0,161.69,SC
3,6220016000,Aerógrafo com Acessórios 16 peças VONDER,314.27,5.0,0.00,50.0,329.98,2.0,313.48,SC
4,6220010000,Aerógrafo com Minicompressor VONDER,307.66,5.0,0.00,50.0,323.04,50.0,306.87,SC


###Bloco Custo Final e Mercado Livre

In [ ]:
#Importa o template Descontos fornecedor
dfdescontos = pd.read_excel('/content/Descontos_fornecedor_'+ timestr + '.xls')

#Importa o template Descontos PAC
dfPAC = pd.read_excel('/content/Descontos_PAC_'+ timestr + '.xls')

#Importa o template Black
dfBlack = pd.read_excel('/content/Descontos_Black_'+ timestr + '.xls')

#Convertendo campos vázios por 0
dfdescontos['DESCONTO'] = dfdescontos.apply(lambda row: changeData2(row['DESCONTO']), axis=1)
dfPAC['DESCONTO'] = dfPAC.apply(lambda row: changeData2(row['DESCONTO']), axis=1)
dfCustoFinal['ESTOQUE Compara'] = dfCustoFinal.apply(lambda row: changeData2(row['ESTOQUE Compara']), axis=1)

#Obtendo desconto fornecedor
dfCustoFinal = pd.merge(dfCustoFinal, dfdescontos[['SKU','DESCONTO']], on='SKU', how='left')
dfCustoFinal['DESCONTO'] = dfCustoFinal.apply(lambda row: changeData2(row['DESCONTO']), axis=1)
dfCustoFinal.rename(columns={'DESCONTO':'DESCONTO FORNECEDOR'}, inplace=True)

#Obtendo desconto PAC
dfCustoFinal = pd.merge(dfCustoFinal, dfPAC[['SKU','DESCONTO']], on='SKU', how='left')
dfCustoFinal['DESCONTO'] = dfCustoFinal.apply(lambda row: changeData2(row['DESCONTO']), axis=1)
dfCustoFinal.rename(columns={'DESCONTO':'DESCONTO PAC'}, inplace=True)

#Obtendo desconto Black
dfCustoFinal = pd.merge(dfCustoFinal, dfBlack[['SKU','DESCONTO']], on='SKU', how='left')
dfCustoFinal['DESCONTO'] = dfCustoFinal.apply(lambda row: changeData2(row['DESCONTO']), axis=1)
dfCustoFinal.rename(columns={'DESCONTO':'DESCONTO BLACK'}, inplace=True)

#Calculando custo final
dfCustoFinal['Custo Final'] = round(dfCustoFinal.apply(lambda row: calculateCostBlack(row['Menor Custo'],row['DESCONTO FORNECEDOR'],row['DESCONTO PAC'], row['DESCONTO BLACK']), axis=1),2)

#Precificando Produtos nos Marketplaces
dfCustoFinal['Site'] = round(dfCustoFinal.apply(lambda row: precoSite(row['Custo Final']), axis=1),2)
dfCustoFinal['Class'] = round(dfCustoFinal.apply(lambda row: precoMLClass(row['Custo Final']), axis=1),2)
dfCustoFinal['Premium'] = round(dfCustoFinal.apply(lambda row: precoMLPremium(row['Class']), axis=1),2)

#Gerando Custo
dfCustoFinal.drop('CUSTO', axis=1, inplace=True)
dfCustoFinal.drop('ESTOQUE', axis=1, inplace=True)
dfCustoFinal.drop('Menor Custo', axis=1, inplace=True)
dfCustoFinal.rename(columns={'CUSTO IMPOSTO':'Custo', 'ESTOQUE Compara':'Estoque'}, inplace=True)

dfCustoFinal.head()

,SKU,Descrição,IPI,%ST,Custo,Estoque,Estado,DESCONTO FORNECEDOR,DESCONTO PAC,DESCONTO BLACK,Custo Final,Site,Class,Premium
0,6659036055,"Acabadora de Superfície a Gasolina, 36"", 7,8 H...",0.0,0.00,6639.27,2.0,PR,0.0,0.0,0.0,6639.27,8963.01,9693.33,10468.80
1,6061350000,Acessórios Para Microretífica Arv350 VONDER,8.0,19.08,220.90,50.0,SC,0.1,0.0,0.0,154.88,209.09,226.12,244.21
2,6220013000,Aerógrafo com Acessórios 13 Peças VONDER,5.0,0.00,170.20,50.0,SC,0.0,0.0,0.0,161.69,218.28,236.07,254.96
3,6220016000,Aerógrafo com Acessórios 16 peças VONDER,5.0,0.00,329.98,2.0,SC,0.0,0.0,0.0,313.48,423.20,457.68,494.29
4,6220010000,Aerógrafo com Minicompressor VONDER,5.0,0.00,323.04,50.0,SC,0.0,0.0,0.0,306.87,414.27,448.03,483.87


###BLOCO B2W

In [ ]:
#Importa o template Frete B2W
dfB2W = pd.read_excel('/content/dados_produtos_'+ timestr + '.xls')

#Pegando o valor atualizado
dfB2W = pd.merge(dfB2W, dfCustoFinal[['SKU','Premium']], on='SKU', how='left')

#Gera a soma das dimensões e o peso cubado
dfB2W['SOMA DIMENSÕES'] = round(dfB2W.apply(lambda row: takeDimensoes(row['LARGURA'], row['ALTURA'], row['COMPRIMENTO']),axis=1),2)
dfB2W['PESO CUBADO'] = round(dfB2W.apply(lambda row: takeCubo(row['LARGURA'], row['ALTURA'], row['COMPRIMENTO']),axis=1),2)

#Convertendo campos vázios em 0
dfB2W['LARGURA'] = dfB2W.apply(lambda row: changeData2(row['LARGURA']), axis=1)
dfB2W['ALTURA'] = dfB2W.apply(lambda row: changeData2(row['ALTURA']), axis=1)
dfB2W['COMPRIMENTO'] = dfB2W.apply(lambda row: changeData2(row['COMPRIMENTO']), axis=1)
dfB2W['Premium'] = dfB2W.apply(lambda row: changeData2(row['Premium']), axis=1)
dfB2W['SOMA DIMENSÕES'] = dfB2W.apply(lambda row: changeData2(row['SOMA DIMENSÕES']), axis=1)
dfB2W['PESO CUBADO'] = dfB2W.apply(lambda row: changeData2(row['PESO CUBADO']), axis=1)

#Calcula o frete da B2W
dfB2W['FreteB2W'] = dfB2W.apply(lambda row: calculaFreteB2W(row['LARGURA'], row['ALTURA'], row['COMPRIMENTO'],row['PESO'], row['SOMA DIMENSÕES'], row['PESO CUBADO'], row['Premium']),axis=1)

#Calcula o preço de venda da B2W
dfB2W['B2W'] = dfB2W.apply(lambda row: precoB2W(row['Premium'], row['FreteB2W']),axis=1)

#Gera a tabela Mestre
dfCustoFinal = pd.merge(dfCustoFinal, dfB2W[['SKU','FreteB2W', 'B2W']], on='SKU', how='left')
dfCustoFinal.drop_duplicates(inplace=True)
#dfSC = pd.merge(dfSC, dfB2W_SC[['SKU','B2W']], on='SKU', how='left')
dfCustoFinal['Custo'] = dfCustoFinal.apply(lambda row: changeData2(row['Custo']), axis=1)
dfCustoFinal['IPI'] = dfCustoFinal.apply(lambda row: changeData2(row['IPI']), axis=1)
dfCustoFinal['FreteB2W'] = dfCustoFinal.apply(lambda row: changeData2(row['FreteB2W']), axis=1)
dfCustoFinal['B2W'] = dfCustoFinal.apply(lambda row: changeData2(row['B2W']), axis=1)
dfCustoFinal.head()

,SKU,Descrição,IPI,%ST,Custo,Estoque,Estado,DESCONTO FORNECEDOR,DESCONTO PAC,DESCONTO BLACK,Custo Final,Site,Class,Premium,FreteB2W,B2W
0,6659036055,"Acabadora de Superfície a Gasolina, 36"", 7,8 H...",0.0,0.00,6639.27,2.0,PR,0.0,0.0,0.0,6639.27,8963.01,9693.33,10468.80,5.0,10473.80
1,6061350000,Acessórios Para Microretífica Arv350 VONDER,8.0,19.08,220.90,50.0,SC,0.1,0.0,0.0,154.88,209.09,226.12,244.21,35.9,280.11
2,6220013000,Aerógrafo com Acessórios 13 Peças VONDER,5.0,0.00,170.20,50.0,SC,0.0,0.0,0.0,161.69,218.28,236.07,254.96,35.9,290.86
3,6220016000,Aerógrafo com Acessórios 16 peças VONDER,5.0,0.00,329.98,2.0,SC,0.0,0.0,0.0,313.48,423.20,457.68,494.29,35.9,530.19
4,6220010000,Aerógrafo com Minicompressor VONDER,5.0,0.00,323.04,50.0,SC,0.0,0.0,0.0,306.87,414.27,448.03,483.87,36.9,520.77


In [ ]:
#Produtos B2W Entregas ou não
dfB2W['Tipo de Envio'] = dfB2W.apply(lambda row: heavierWeight(row['PESO'], row['PESO CUBADO'], row['SOMA DIMENSÕES'], row['LARGURA'], row['COMPRIMENTO'], row['ALTURA'], row['B2W']),axis=1)
dfB2W.drop(dfB2W[dfB2W['B2W'].isnull()].index, inplace=True)
dfB2W.to_excel('Produtos pesados.xls', index=False)

###Relatório Tabela Mestre

In [ ]:
#Custo sem a B2W
dfCustoFinal.rename(columns={'DESCRIÇÃO':'Descrição', 'DESCONTO FORNECEDOR':'Desconto Fornecedor', 'DESCONTO PAC':'Desconto PAC', 'DESCONTO BLACK':'Desconto Black', 'FreteB2W':'Frete B2W' }, inplace=True)
dfPrecoFinal = pd.DataFrame(dfCustoFinal[['SKU', 'Descrição', 'Estoque', 'Custo', 'IPI', '%ST', 'Desconto Fornecedor', 'Desconto PAC', 'Desconto Black', 'Custo Final', 'Site', 'Class', 'Premium', 'Estado']])
dfPrecoFinal.to_excel('Mestre V'+ timestr+'.xls', index=False)

In [ ]:
#Custo com a B2W - Rodar célula da B2W antes
dfCustoFinal.rename(columns={'DESCRIÇÃO':'Descrição', 'ESTOQUE':'Estoque', 'DESCONTO FORNECEDOR':'Desconto Fornecedor', 'DESCONTO PAC':'Desconto PAC', 'DESCONTO BLACK':'Desconto Black', 'FreteB2W':'Frete B2W' }, inplace=True)
dfCustoFinal = pd.DataFrame(dfCustoFinal[['SKU', 'Descrição', 'Estoque', 'Custo', 'IPI', '%ST', 'Desconto Fornecedor', 'Desconto PAC', 'Desconto Black', 'Custo Final', 'Site', 'Class', 'Premium','Frete B2W','B2W', 'Estado']])
dfCustoFinal.to_excel('Mestre V'+ timestr+'_B2W.xls', index=False)

###Lista de Preço B2W

In [ ]:
dfCustoFinal.rename(columns={'SKU':'Código (SKU)','DESCRIÇÃO':'Descrição','Premium':'Preço', 'B2W':'Preço Promocional'}, inplace=True)

In [ ]:
dfListaPreçoB2W = dfCustoFinal[['Código (SKU)', 'Descrição', 'Preço', 'Preço Promocional']]
dfListaPreçoB2W.loc['Preço Promocional'] = dfListaPreçoB2W['Preço']
dfListaPreçoB2W.to_excel('lista_preco_'+ timestr + '.xls', index=False, encoding='latin1')
dfListaPreçoB2W.head()

##Preço e estoque PR e SC

###Produtos SC

In [ ]:
dfprecoSC1 = pd.read_csv('/content/produtos_2021-10-04-13-35-10.csv', sep=';', encoding='latin1')
dfprecoSC2 = pd.read_csv('/content/produtos_2021-10-04-13-35-27.csv', sep=';', encoding='latin1')
dfprecoSC3 = pd.read_csv('/content/produtos_2021-10-04-13-35-34.csv', sep=';', encoding='latin1')

dfprecoSC1.drop(dfprecoSC1.loc[dfprecoSC1['Marca'] != 'Vonder'].index, inplace=True)
dfprecoSC2.drop(dfprecoSC2.loc[dfprecoSC2['Marca'] != 'Vonder'].index, inplace=True)
dfprecoSC3.drop(dfprecoSC3.loc[dfprecoSC3['Marca'] != 'Vonder'].index, inplace=True)

dfprecoSC1.rename(columns={'Código (SKU)':'SKU','Estoque':'Estoque1'}, inplace=True)
dfprecoSC2.rename(columns={'Código (SKU)':'SKU','Estoque':'Estoque1'}, inplace=True)
dfprecoSC3.rename(columns={'Código (SKU)':'SKU','Estoque':'Estoque1'}, inplace=True)

dfprecoSC1['SKU'] = dfprecoSC1['SKU'].astype(int)
dfprecoSC2['SKU'] = dfprecoSC2['SKU'].astype(int)
dfprecoSC3['SKU'] = dfprecoSC3['SKU'].astype(int)

In [ ]:
dfprecoSC1 = pd.merge(dfprecoSC1, dfCustoFinal[['SKU', 'Estoque', 'Custo Final', 'Site']], how='left', on=['SKU'] )
dfprecoSC2 = pd.merge(dfprecoSC2, dfCustoFinal[['SKU', 'Estoque', 'Custo Final', 'Site']], how='left', on=['SKU'] )
dfprecoSC3 = pd.merge(dfprecoSC3, dfCustoFinal[['SKU', 'Estoque', 'Custo Final', 'Site']], how='left', on=['SKU'] )

dfprecoSC1.drop('Preço', axis=1, inplace=True)
dfprecoSC1.drop('Preço de custo', axis=1, inplace=True)

dfprecoSC2.drop('Preço', axis=1, inplace=True)
dfprecoSC2.drop('Preço de custo', axis=1, inplace=True)

dfprecoSC3.drop('Preço', axis=1, inplace=True)
dfprecoSC3.drop('Preço de custo', axis=1, inplace=True)

dfprecoSC1.rename(columns={'SKU':'Código (SKU)','Site':'Preço','Custo Final':'Preço de custo'}, inplace=True)
dfprecoSC2.rename(columns={'SKU':'Código (SKU)','Site':'Preço','Custo Final':'Preço de custo'}, inplace=True)
dfprecoSC3.rename(columns={'SKU':'Código (SKU)','Site':'Preço','Custo Final':'Preço de custo'}, inplace=True)

dfprecoSC1['Preço'] = dfprecoSC1.apply(lambda row: changeData2(row['Preço']), axis=1)
dfprecoSC1['Preço de custo'] = dfprecoSC1.apply(lambda row: changeData2(row['Preço de custo']), axis=1)
dfprecoSC1['Estoque'] = dfprecoSC1.apply(lambda row: changeData2(row['Estoque']), axis=1)

dfprecoSC2['Preço'] = dfprecoSC2.apply(lambda row: changeData2(row['Preço']), axis=1)
dfprecoSC2['Preço de custo'] = dfprecoSC2.apply(lambda row: changeData2(row['Preço de custo']), axis=1)
dfprecoSC2['Estoque'] = dfprecoSC2.apply(lambda row: changeData2(row['Estoque']), axis=1)

dfprecoSC3['Preço'] = dfprecoSC3.apply(lambda row: changeData2(row['Preço']), axis=1)
dfprecoSC3['Preço de custo'] = dfprecoSC3.apply(lambda row: changeData2(row['Preço de custo']), axis=1)
dfprecoSC3['Estoque'] = dfprecoSC3.apply(lambda row: changeData2(row['Estoque']), axis=1)

strtimeSC = time.strftime('%Y-%m-%d-%H-%M-%S')

In [ ]:
PrecoSC1 = pd.DataFrame(dfprecoSC1[['ID',	'Código (SKU)',	'Descrição',	'Unidade',	'Classificação fiscal',	'Origem',	'Preço',	'Valor IPI fixo',	'Observações',	'Situação',	'Estoque',	'Preço de custo',	'Cód do fabricante',	'Fabricante',	'Localização',	'Estoque máximo',	'Estoque mínimo',	'Peso líquido (Kg)',	'Peso bruto (Kg)',	'GTIN/EAN',	'GTIN/EAN tributável',	'Descrição complementar',	'CEST',	'Código de Enquadramento IPI',	'Formato embalagem',	'Largura embalagem',	'Altura embalagem',	'Comprimento embalagem',	'Diâmetro embalagem',	'Tipo do produto',	'URL imagem 1',	'URL imagem 2',	'URL imagem 3',	'URL imagem 4',	'URL imagem 5',	'URL imagem 6',	'Categoria',	'Código do pai',	'Variações',	'Marca',	'Garantia',	'Sob encomenda',	'Preço promocional',	'URL imagem externa 1',	'URL imagem externa 2',	'URL imagem externa 3',	'URL imagem externa 4',	'URL imagem externa 5',	'URL imagem externa 6',	'Link do vídeo',	'Título SEO',	'Descrição SEO',	'Palavras chave SEO',	'Slug',	'Dias para preparação',	'Controlar lotes',	'Unidade por caixa']])
PrecoSC2 = pd.DataFrame(dfprecoSC2[['ID',	'Código (SKU)',	'Descrição',	'Unidade',	'Classificação fiscal',	'Origem',	'Preço',	'Valor IPI fixo',	'Observações',	'Situação',	'Estoque',	'Preço de custo',	'Cód do fabricante',	'Fabricante',	'Localização',	'Estoque máximo',	'Estoque mínimo',	'Peso líquido (Kg)',	'Peso bruto (Kg)',	'GTIN/EAN',	'GTIN/EAN tributável',	'Descrição complementar',	'CEST',	'Código de Enquadramento IPI',	'Formato embalagem',	'Largura embalagem',	'Altura embalagem',	'Comprimento embalagem',	'Diâmetro embalagem',	'Tipo do produto',	'URL imagem 1',	'URL imagem 2',	'URL imagem 3',	'URL imagem 4',	'URL imagem 5',	'URL imagem 6',	'Categoria',	'Código do pai',	'Variações',	'Marca',	'Garantia',	'Sob encomenda',	'Preço promocional',	'URL imagem externa 1',	'URL imagem externa 2',	'URL imagem externa 3',	'URL imagem externa 4',	'URL imagem externa 5',	'URL imagem externa 6',	'Link do vídeo',	'Título SEO',	'Descrição SEO',	'Palavras chave SEO',	'Slug',	'Dias para preparação',	'Controlar lotes',	'Unidade por caixa']])
PrecoSC3 = pd.DataFrame(dfprecoSC3[['ID',	'Código (SKU)',	'Descrição',	'Unidade',	'Classificação fiscal',	'Origem',	'Preço',	'Valor IPI fixo',	'Observações',	'Situação',	'Estoque',	'Preço de custo',	'Cód do fabricante',	'Fabricante',	'Localização',	'Estoque máximo',	'Estoque mínimo',	'Peso líquido (Kg)',	'Peso bruto (Kg)',	'GTIN/EAN',	'GTIN/EAN tributável',	'Descrição complementar',	'CEST',	'Código de Enquadramento IPI',	'Formato embalagem',	'Largura embalagem',	'Altura embalagem',	'Comprimento embalagem',	'Diâmetro embalagem',	'Tipo do produto',	'URL imagem 1',	'URL imagem 2',	'URL imagem 3',	'URL imagem 4',	'URL imagem 5',	'URL imagem 6',	'Categoria',	'Código do pai',	'Variações',	'Marca',	'Garantia',	'Sob encomenda',	'Preço promocional',	'URL imagem externa 1',	'URL imagem externa 2',	'URL imagem externa 3',	'URL imagem externa 4',	'URL imagem externa 5',	'URL imagem externa 6',	'Link do vídeo',	'Título SEO',	'Descrição SEO',	'Palavras chave SEO',	'Slug',	'Dias para preparação',	'Controlar lotes',	'Unidade por caixa']])

In [ ]:
PrecoSC1.to_excel('produtos_SC_' + strtimeSC + '_1.xls', index=False)
PrecoSC2.to_excel('produtos_SC_' + strtimeSC + '_2.xls', index=False)
PrecoSC3.to_excel('produtos_SC_' + strtimeSC + '_3.xls', index=False)

###Produtos PR

In [ ]:
#Importando bases com o base dos cadastros
dfprecoPR1 = pd.read_csv('/content/produtos_2021-10-04-13-31-51.csv', sep=';', encoding='latin1')
dfprecoPR2 = pd.read_csv('/content/produtos_2021-10-04-13-32-15.csv', sep=';', encoding='latin1')
dfprecoPR3 = pd.read_csv('/content/produtos_2021-10-04-13-32-22.csv', sep=';', encoding='latin1')

#Filtrando apenas Vonder
dfprecoPR1.drop(dfprecoPR1.loc[dfprecoPR1['Marca'] != 'Vonder'].index, inplace=True)
dfprecoPR2.drop(dfprecoPR2.loc[dfprecoPR2['Marca'] != 'Vonder'].index, inplace=True)
dfprecoPR3.drop(dfprecoPR3.loc[dfprecoPR3['Marca'] != 'Vonder'].index, inplace=True)

#Alterando nomes para realização de um merge
dfprecoPR1.rename(columns={'Código (SKU)':'SKU','Estoque':'Estoque1'}, inplace=True)
dfprecoPR2.rename(columns={'Código (SKU)':'SKU','Estoque':'Estoque1'}, inplace=True)
dfprecoPR3.rename(columns={'Código (SKU)':'SKU','Estoque':'Estoque1'}, inplace=True)

In [ ]:
#Convertendo coluna para realização de um merge
dfprecoPR1['SKU'] = dfprecoPR1['SKU'].astype(int)
dfprecoPR2['SKU'] = dfprecoPR2['SKU'].astype(int)
dfprecoPR3['SKU'] = dfprecoPR3['SKU'].astype(int)

#Atualizando Estoque, custo e preço das tabelas bases
dfprecoPR1 = pd.merge(dfprecoPR1, dfCustoFinal[['SKU', 'Estoque', 'Custo Final', 'Site']], how='left', on=['SKU'] )
dfprecoPR2 = pd.merge(dfprecoPR2, dfCustoFinal[['SKU', 'Estoque', 'Custo Final', 'Site']], how='left', on=['SKU'] )
dfprecoPR3 = pd.merge(dfprecoPR3, dfCustoFinal[['SKU', 'Estoque', 'Custo Final', 'Site']], how='left', on=['SKU'] )

In [ ]:
#Eliminando colunas desnecessárias
dfprecoPR1.drop('Preço', axis=1, inplace=True)
dfprecoPR1.drop('Preço de custo', axis=1, inplace=True)
dfprecoPR2.drop('Preço', axis=1, inplace=True)
dfprecoPR2.drop('Preço de custo', axis=1, inplace=True)
dfprecoPR3.drop('Preço', axis=1, inplace=True)
dfprecoPR3.drop('Preço de custo', axis=1, inplace=True)

#Renomeando as colunas conforme dataset original
dfprecoPR1.rename(columns={'SKU':'Código (SKU)','Site':'Preço','Custo Final':'Preço de custo'}, inplace=True)
dfprecoPR2.rename(columns={'SKU':'Código (SKU)','Site':'Preço','Custo Final':'Preço de custo'}, inplace=True)
dfprecoPR3.rename(columns={'SKU':'Código (SKU)','Site':'Preço','Custo Final':'Preço de custo'}, inplace=True)

#Preenchendo colunas vazias
dfprecoPR1['Preço'] = dfprecoPR1.apply(lambda row: changeData2(row['Preço']), axis=1)
dfprecoPR1['Preço de custo'] = dfprecoPR1.apply(lambda row: changeData2(row['Preço de custo']), axis=1)
dfprecoPR1['Estoque'] = dfprecoPR1.apply(lambda row: changeData2(row['Estoque']), axis=1)

dfprecoPR2['Preço'] = dfprecoPR2.apply(lambda row: changeData2(row['Preço']), axis=1)
dfprecoPR2['Preço de custo'] = dfprecoPR2.apply(lambda row: changeData2(row['Preço de custo']), axis=1)
dfprecoPR2['Estoque'] = dfprecoPR2.apply(lambda row: changeData2(row['Estoque']), axis=1)

dfprecoPR3['Preço'] = dfprecoPR3.apply(lambda row: changeData2(row['Preço']), axis=1)
dfprecoPR3['Preço de custo'] = dfprecoPR3.apply(lambda row: changeData2(row['Preço de custo']), axis=1)
dfprecoPR3['Estoque'] = dfprecoPR3.apply(lambda row: changeData2(row['Estoque']), axis=1)

#Criando uma nova planilhas com os dados atualizados
PrecoPR1 = pd.DataFrame(dfprecoPR1[['ID',	'Código (SKU)',	'Descrição',	'Unidade',	'Classificação fiscal',	'Origem',	'Preço',	'Valor IPI fixo',	'Observações',	'Situação',	'Estoque',	'Preço de custo',	'Cód do fabricante',	'Fabricante',	'Localização',	'Estoque máximo',	'Estoque mínimo',	'Peso líquido (Kg)',	'Peso bruto (Kg)',	'GTIN/EAN',	'GTIN/EAN tributável',	'Descrição complementar',	'CEST',	'Código de Enquadramento IPI',	'Formato embalagem',	'Largura embalagem',	'Altura embalagem',	'Comprimento embalagem',	'Diâmetro embalagem',	'Tipo do produto',	'URL imagem 1',	'URL imagem 2',	'URL imagem 3',	'URL imagem 4',	'URL imagem 5',	'URL imagem 6',	'Categoria',	'Código do pai',	'Variações',	'Marca',	'Garantia',	'Sob encomenda',	'Preço promocional',	'URL imagem externa 1',	'URL imagem externa 2',	'URL imagem externa 3',	'URL imagem externa 4',	'URL imagem externa 5',	'URL imagem externa 6',	'Link do vídeo',	'Título SEO',	'Descrição SEO',	'Palavras chave SEO',	'Slug',	'Dias para preparação',	'Controlar lotes',	'Unidade por caixa']])
PrecoPR2 = pd.DataFrame(dfprecoPR2[['ID',	'Código (SKU)',	'Descrição',	'Unidade',	'Classificação fiscal',	'Origem',	'Preço',	'Valor IPI fixo',	'Observações',	'Situação',	'Estoque',	'Preço de custo',	'Cód do fabricante',	'Fabricante',	'Localização',	'Estoque máximo',	'Estoque mínimo',	'Peso líquido (Kg)',	'Peso bruto (Kg)',	'GTIN/EAN',	'GTIN/EAN tributável',	'Descrição complementar',	'CEST',	'Código de Enquadramento IPI',	'Formato embalagem',	'Largura embalagem',	'Altura embalagem',	'Comprimento embalagem',	'Diâmetro embalagem',	'Tipo do produto',	'URL imagem 1',	'URL imagem 2',	'URL imagem 3',	'URL imagem 4',	'URL imagem 5',	'URL imagem 6',	'Categoria',	'Código do pai',	'Variações',	'Marca',	'Garantia',	'Sob encomenda',	'Preço promocional',	'URL imagem externa 1',	'URL imagem externa 2',	'URL imagem externa 3',	'URL imagem externa 4',	'URL imagem externa 5',	'URL imagem externa 6',	'Link do vídeo',	'Título SEO',	'Descrição SEO',	'Palavras chave SEO',	'Slug',	'Dias para preparação',	'Controlar lotes',	'Unidade por caixa']])
PrecoPR3 = pd.DataFrame(dfprecoPR3[['ID',	'Código (SKU)',	'Descrição',	'Unidade',	'Classificação fiscal',	'Origem',	'Preço',	'Valor IPI fixo',	'Observações',	'Situação',	'Estoque',	'Preço de custo',	'Cód do fabricante',	'Fabricante',	'Localização',	'Estoque máximo',	'Estoque mínimo',	'Peso líquido (Kg)',	'Peso bruto (Kg)',	'GTIN/EAN',	'GTIN/EAN tributável',	'Descrição complementar',	'CEST',	'Código de Enquadramento IPI',	'Formato embalagem',	'Largura embalagem',	'Altura embalagem',	'Comprimento embalagem',	'Diâmetro embalagem',	'Tipo do produto',	'URL imagem 1',	'URL imagem 2',	'URL imagem 3',	'URL imagem 4',	'URL imagem 5',	'URL imagem 6',	'Categoria',	'Código do pai',	'Variações',	'Marca',	'Garantia',	'Sob encomenda',	'Preço promocional',	'URL imagem externa 1',	'URL imagem externa 2',	'URL imagem externa 3',	'URL imagem externa 4',	'URL imagem externa 5',	'URL imagem externa 6',	'Link do vídeo',	'Título SEO',	'Descrição SEO',	'Palavras chave SEO',	'Slug',	'Dias para preparação',	'Controlar lotes',	'Unidade por caixa']])

strtime = time.strftime('%Y-%m-%d-%H-%M-%S')

In [ ]:
PrecoPR1.to_excel('produtos_PR_' + strtime + '_1.xls', index=False)
PrecoPR2.to_excel('produtos_PR_' + strtime + '_2.xls', index=False)
PrecoPR3.to_excel('produtos_PR_' + strtime + '_3.xls', index=False)